Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from queue import PriorityQueue
from math import ceil

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [4]:
#generate the problems so that they are stored in memory
points = [100, 1_000, 5_000]
densities = [.3, .7]

problems = []
for num_points in points:
    for density in densities:
        num_sets = num_points
        sets = make_set_covering_problem(num_points, num_sets, density)
        print('problem with', sets.nnz, 'non zero values:', num_points, density)
        problems.append(sets)

problem with 3066 non zero values: 100 0.3
problem with 7019 non zero values: 100 0.7
problem with 300776 non zero values: 1000 0.3
problem with 701016 non zero values: 1000 0.7
problem with 7500703 non zero values: 5000 0.3
problem with 17508823 non zero values: 5000 0.7


In [5]:
solution = [1,2,3,4]  #store indices of taken sets to form solution, initialized randomly
a = problems[0].getrow(0)

min(problems[0][:, :].sum(axis = 0)) #dtype = bool


19

# Solve one problem at a time, try to solve them
 we use slicing cos its fast (from scipy docs)

In [6]:
def fitness(sets, solution):
    return ((sets[solution, :].sum(axis = 0, dtype = bool).sum() - sets.shape[0]), sets.shape[0] - len(solution)) 

P = 0.5
def tweak1(sets, solution):                                                      
    mutation = copy(solution)
    if len(mutation) == 0 or random() <= P:
        mutation.append(randint(0, sets.shape[0]-1))
    else:
        mutation[randint(0, len(mutation) - 1)] = randint(0, sets.shape[0]-1)    
    return mutation

def tweak2(sets, solution):                                                      #improve: needs to be a set
    mutation = copy(solution)
    if len(mutation) == 0 or random() <= P:
        mutation.append(randint(0, sets.shape[0]-1))
    else:
        mutation.pop(randint(0, len(mutation) - 1))    
    return mutation

tweak = tweak2


In [7]:
MESA_THRESHOLD = 150

for sets in problems:
    solution = []
    fitness_prev = fitness(sets, solution)
    since_last_mutation = 0
    it = 0
    print(fitness_prev, '->', end=' ')

    while(since_last_mutation < MESA_THRESHOLD):
        it += 1
        mutation = tweak(sets, solution)
        fitness_new = fitness(sets, mutation)
        if fitness_new >= fitness_prev:
            fitness_prev = fitness_new
            solution = mutation
            since_last_mutation = 0
        else:
            since_last_mutation += 1    
    print(fitness_prev, '\n', solution)
    print('called fitness function', it, 'times', end = '\n\n')

(-100, 100) -> (0, 92) 
 [63, 19, 20, 17, 42, 84, 30, 6]
called fitness function 204 times

(-100, 100) -> (0, 95) 
 [74, 45, 87, 68, 78]
called fitness function 170 times

(-1000, 1000) -> (0, 986) 
 [458, 17, 374, 43, 240, 507, 215, 552, 202, 670, 881, 141, 161, 163]
called fitness function 233 times

(-1000, 1000) -> (0, 994) 
 [655, 193, 905, 682, 15, 352]
called fitness function 164 times

(-5000, 5000) -> (0, 4982) 
 [3010, 937, 2296, 2929, 2995, 2584, 2468, 1237, 3755, 2672, 1864, 410, 358, 4153, 3300, 4226, 4736, 580]
called fitness function 247 times

(-5000, 5000) -> (0, 4993) 
 [3726, 341, 2114, 3087, 3520, 495, 1482]
called fitness function 169 times



# Try a more 'greedy' approach

In [8]:
for sets in problems:
    solution = []
    fitness_prev = fitness(sets, solution)
    print(fitness_prev, '->', end=' ')

    while(fitness_prev[0] != 0):
        it += 1
        mutation = tweak(sets, solution)
        fitness_new = fitness(sets, mutation)
        if fitness_new >= fitness_prev:
            fitness_prev = fitness_new
            solution = mutation
               
    print(fitness_prev, '\n', solution)
    print('called fitness function', it, 'times', end = '\n\n')

(-100, 100) -> (0, 91) 
 [86, 11, 73, 58, 71, 39, 9, 41, 17]
called fitness function 208 times

(-100, 100) -> (0, 95) 
 [72, 53, 0, 98, 40]
called fitness function 219 times

(-1000, 1000) -> (0, 984) 
 [560, 441, 806, 855, 103, 956, 814, 218, 707, 18, 240, 766, 716, 362, 565, 831]
called fitness function 296 times

(-1000, 1000) -> (0, 993) 
 [148, 635, 809, 612, 244, 63, 494]
called fitness function 312 times

(-5000, 5000) -> (0, 4979) 
 [4954, 137, 1825, 1793, 4083, 2064, 2179, 4283, 4573, 3641, 1913, 26, 1989, 2167, 4317, 4977, 2006, 4818, 3535, 1246, 115]
called fitness function 354 times

(-5000, 5000) -> (0, 4992) 
 [4031, 3174, 2479, 2632, 1372, 2860, 3187, 3406]
called fitness function 369 times



# 1 + lambda ES
 takes 3 different paths, one additive, one subtractive, one that swaps 

In [9]:
# 1 + lambda with different strategies
# already returns best solution and its fitness
def tweak3(sets, solution): 
    mutation1 = copy(solution)
    mutation1.append(randint(0, sets.shape[0]-1))
    f1 = fitness(sets, mutation1)
    if len(mutation1) == 1:
        return mutation1, f1
    
    mutation2 = copy(solution)    
    mutation2[randint(0, len(mutation2) - 1)] = randint(0, sets.shape[0]-1) 
    f2 = fitness(sets, mutation2)

    mutation3 = copy(solution)
    mutation3.pop(randint(0, len(mutation3) - 1)) 
    f3 = fitness(sets, mutation3)

    if(f1 >= f2 and f1 >= f3):
        return mutation1, f1
    if(f2 >= f1 and f2 >= f3):
        return mutation2, f2  
    return mutation3, f3

tweak = tweak3

for sets in problems:
    solution = []
    fitness_prev = fitness(sets, solution)
    it = 0
    print(fitness_prev, '->', end=' ')

    while(fitness_prev[0] != 0):
        it += 1
        mutation, fitness_new = tweak(sets, solution)
        if fitness_new >= fitness_prev:
            fitness_prev = fitness_new
            solution = mutation  
    print(fitness_prev, '\n', solution)
    print('called fitness function', it*3, 'times', end = '\n\n')

(-100, 100) -> (0, 90) 
 [15, 97, 10, 25, 18, 91, 92, 80, 57, 68]
called fitness function 36 times

(-100, 100) -> (0, 95) 
 [93, 82, 64, 91, 71]
called fitness function 15 times

(-1000, 1000) -> (0, 983) 
 [844, 742, 784, 417, 746, 234, 516, 422, 792, 31, 544, 302, 126, 365, 997, 243, 988]
called fitness function 57 times

(-1000, 1000) -> (0, 994) 
 [58, 740, 249, 789, 6, 501]
called fitness function 21 times

(-5000, 5000) -> (0, 4980) 
 [102, 4294, 4536, 178, 4386, 1164, 338, 3284, 313, 2271, 2245, 844, 3789, 1933, 3876, 3989, 4431, 4357, 4597, 4353]
called fitness function 63 times

(-5000, 5000) -> (0, 4993) 
 [3199, 1338, 4875, 493, 3044, 3553, 1240]
called fitness function 21 times



## Now try to make a stupid - but fast - algorithm

In [12]:

def sorted_sets(sets):
    tile_rarities = sets.sum(axis = 0)
    sets_richness = sets.sum(axis = 1)
    #i like if if: lot of tiles, tiles are rare -> small number ( sum of sets that have that tile / number of tiles )
    strengths = []
    for i in range(sets.shape[0]):
        strengths.append( (sets[[i], :] * tile_rarities).sum() / sets_richness[i])
    return sorted(list(range(sets.shape[0])), key = lambda i: strengths[i])
    

for sets in problems:
    s_sorted = sorted_sets(sets)

    take = ceil(sets.shape[0] / sets[[s_sorted[0]], :].sum())
    solution = s_sorted[0 : take]
    fitness_prev = fitness(sets, solution)
    it = 0
    print(fitness_prev, '->', end=' ')

    while(fitness_prev[0] != 0):
        solution = s_sorted[0 : 2**it + take]
        fitness_prev = fitness(sets, solution)
        it += 1      
    print(fitness_prev, '\n', solution)
    print('called fitness function', it, 'times', end = '\n\n')
    


(-30, 96) -> (0, 80) 
 [80, 92, 22, 94, 36, 58, 0, 56, 90, 91, 7, 43, 44, 96, 82, 42, 64, 29, 37, 61]
called fitness function 5 times

(-10, 98) -> (0, 94) 
 [13, 81, 2, 22, 96, 47]
called fitness function 3 times

(-236, 996) -> (0, 964) 
 [229, 516, 991, 157, 542, 89, 695, 427, 12, 97, 791, 403, 720, 244, 81, 824, 310, 357, 470, 36, 346, 25, 509, 199, 663, 124, 687, 820, 640, 935, 61, 814, 462, 376, 860, 583]
called fitness function 6 times

(-69, 998) -> (0, 994) 
 [278, 516, 955, 395, 846, 10]
called fitness function 3 times

(-1208, 4996) -> (0, 4964) 
 [4626, 1933, 2012, 3105, 2478, 1329, 4759, 257, 200, 3562, 2363, 955, 1028, 939, 4422, 2583, 3692, 3021, 3133, 4006, 1035, 124, 1610, 1594, 1725, 1921, 4495, 715, 1365, 2466, 3115, 854, 4031, 4825, 3482, 1073]
called fitness function 6 times

(-441, 4998) -> (0, 4990) 
 [4316, 1007, 1113, 2913, 1553, 1050, 2214, 2830, 4330, 1414]
called fitness function 4 times



In [11]:
b = [4,3,2.4,2.5,1,0]

def brt(index):
    return b[index]

a = list(range(6))
a.sort(key = lambda i:-i)
print((sets[[0], :]).sum())

3555


## optimizing the number of fitness calls
 - something like dijkstra + evolutionary strategies
 - or again, try ordering the sets if it is doable and not sloooow